In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import os
import pytz
from google.cloud import bigquery
from datetime import datetime

In [2]:
timezone = pytz.timezone('US/Eastern')
current_datetime = datetime.now(tz=timezone)

table_name = current_datetime.strftime('%Y%m%d')

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/wzhang/Documents/WorkFile/GoogleAPI/insideradrevenue-3e1aa74bcbfe.json"

In [ ]:
df = []

In [4]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
    'Connection': 'keep-alive',
    'Accept-Encoding':'gzip, deflate, br'
}

In [5]:
s = requests.session()

In [6]:
url_re = s.get('https://www.msn.com', headers=headers)

In [ ]:
if url_re.status_code == 200 :
    soup = BeautifulSoup(url_re.text)
    if (soup.select('img[alt="Business Insider Logo"]')) or (soup.select('img[alt="INSIDER Logo"]')):
        #### main river ####
        for item in soup.select('.todaystripe.grid')[0].select('ul')[0].findAll("li", recursive=False):
            if (item.select('img[alt="Business Insider Logo"]')) or (item.select('img[alt="INSIDER Logo"]')):
                river_position = 1
                #river_name = 'top river'       
                if item['class'][0] == 'infopane-placeholder':
                    river_name = 'carousel'       
                    item_block_position = 1
                    item_block_type = 'swipenav'
                    for item_block_link in item.find(class_='swipenav').findAll("li", recursive=False):
                        if (item_block_link.select('img[alt="Business Insider Logo"]')) or (item_block_link.select('img[alt="INSIDER Logo"]')):
                            item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]

                            for item_block_link_each in item_block_link.select('a'):
                                if (item_block_link_each.select('img[alt="Business Insider Logo"]')) or (item_block_link_each.select('img[alt="INSIDER Logo"]')):
                                    article_title = item_block_link_each.select('span[aria-label]')[0]['aria-label']
                                    article_link = item_block_link_each['href']

                                    try:
                                        article_img = re.findall(r'\/\/(.*)\?',item_block_link_each.select('img')[0]['data-src'])[0]
                                    except:
                                        article_img = item_block_link_each.select('img')[0]['src']
                                    try: 
                                        article_img_small = re.findall('\/\/(.+)\"', item_block_link_each.select('img')[0]['data-src'])[0]
                                    except:
                                        article_img_small = item_block_link_each.select('img')[0]['src']
                                    df.append([river_position, 
                                                   river_name,
                                                   item_block_position,
                                                   item_block_type,
                                                   item_block_link_position,
                                                   article_title,
                                                   article_link,
                                                   article_img,
                                                   article_img_small])

                else:
                    try:
                        river_name = 'above the fold'
                        item_block_position = re.findall("\"o\":(\d+)",item['data-m'])[0]
                        item_block_type = ' '.join(item['class'])
                        item_block_link_position = 1
                        try:
                            article_title = item.select('span[aria-label]')[0]['aria-label']
                        except:
                            article_title = item.find(class_='title')
                        article_link = item.a['href']
                        article_img = re.findall(r'\/\/(.*)\?',item.select('img')[0]['data-src'])[0]
                        article_img_small = re.findall('\/\/(.+)\"', item.a.img['data-src'])[0]
                        df.append([river_position, 
                                                   river_name,
                                                   item_block_position,
                                                   item_block_type,
                                                   item_block_link_position,
                                                   article_title,
                                                   article_link,
                                                   article_img,
                                                   article_img_small])
                    except:
                        print(' '.join(item['class']))





        ### non-top river ###            
        for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):
            if (item.select('img[alt="Business Insider Logo"]')) or item.select('img[alt="INSIDER Logo"]'):
                river_position = index
                river_name = item['aria-label']
                for item_block in item.find_all('ul'):
                    if (item_block.select('img[alt="Business Insider Logo"]')) or item_block.select('img[alt="INSIDER Logo"]'):
                        item_block_type = item_block['data-aop']
                        item_block_position = re.findall("\"o\":(\d+)",item_block['data-m'])[0]
                        for item_block_link in item_block.select('li'):
                            if (item_block_link.select('img[alt="Business Insider Logo"]')) or item_block_link.select('img[alt="INSIDER Logo"]'):
                                item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
                                article_title = item_block_link.select('h3[aria-label]')[0]['aria-label']
                                article_link = item_block_link.a['href']
                                article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                                article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]
                                df.append([river_position, 
                                           river_name,
                                           item_block_position,
                                           item_block_type,
                                           item_block_link_position,
                                           article_title,
                                           article_link,
                                           article_img,
                                           article_img_small])


        df = pd.DataFrame(df, columns = ['river_position',
                                   'river_name',
                                   'item_block_position',
                                   'item_block_type',
                                   'item_block_link_position',
                                   'article_title',
                                   'article_link',
                                   'article_img',
                                   'article_img_small'])
        df['time'] = current_datetime.strftime('%Y%m%d %H:%M')
        df.set_index(df.columns[-1], inplace = True)   #df_bq.set_index(df_bq.columns[-1], inplace = True)

        client = bigquery.Client()
        df['river_position'] = (df['river_position']).astype(int)
        df['item_block_position'] = (df['item_block_position']).astype(int)
        df['item_block_link_position'] = (df['item_block_link_position']).astype(int)


        table_ref = client.dataset("MSN", project='insideradrevenue').table(
                    "MSN_HP_{}".format(table_name))

        job_config = bigquery.LoadJobConfig()
        job_config.write_disposition = 'WRITE_APPEND'
        job_config.autodetect = True
        job = client.load_table_from_dataframe(
                df,
                table_ref,
                location='US',
                job_config=job_config)